In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model

#loading and separating our wine dataset into labels and features
df = pd.read_csv('winequality-red.csv')


In [2]:
df.head()

fixed_acidity  volatile_acidity  citric_acid  residual_sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free_sulfur_dioxide  total_sulfur_dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5  
3      9.8        6  
4      9.4        5

In [3]:
label = df['quality']
features = df.drop('quality', axis=1)

#defining our linear regression estimator and training it with our wine data
regr = linear_model.LinearRegression()
regr.fit(features, label)

#using our trained model to predict a fake wine
#each number represents a feature like pH, acidity, etc.
print (regr.predict([[7.4,0.66,0,1.8,0.075,13,40,0.9978,3.51,0.56,9.4]]).tolist())

[5.065570349074964]


In [4]:
import pickle

In [5]:
pickle.dump(regr, open("model.pkl","wb" , ))



In [16]:
#loading a model from a file called model.pkl
model = pickle.load(open("model.pkl", "rb"))

In [18]:
# Creating web server using flask here. 
# It has two routes defined. One for printing "Hello"
# The other to call the predict() function of the model

from flask import Flask
from flask import request

#code which helps initialize our server
app = Flask(__name__)

#defining a /hello route for only post requests
@app.route('/hello', methods=['POST'])
def index():
    #grabs the data tagged as 'name'
    name = request.get_json()['name']
    
    #sending a hello back to the requester
    return "Hello " + name


# Using pickle to upload the model into memory of the webserver
# Linear models hold only the coefficients and the intercept. All these will be available in the object
# Had this been KNN, then we need to install the database and connnect the model to the dataset as KNN needs entier data

model = pickle.load(open("model.pkl","rb"))    # Deserializing the model from the .pkl file. This should be available on server

@app.route('/predict', methods=['POST'])    # What needs to be done for request to predict
def predict():
    #grabbing a set of wine features from the request's body
    feature_array = request.get_json()['feature_array']
    
    #our model rates the wine based on the input array
    prediction = model.predict([feature_array]).tolist()
    
    #preparing a response object and storing the model's predictions
    response = {}
    response['predictions'] = prediction
    
    #sending our response object back as json
    return flask.jsonify(response)